# Домашнее задание 1 - twitter

### Задание 1

Прочтём файл с данными в список tweets - это список словарей. Узнаем, сколько
всего окажется твитов, посмотрев длину списка. (Их **2556**).

In [71]:
import json
import re
from pprint import pprint
from collections import Counter


file = 'data/hw_3_twitter.json'

tweets = list()
for line in open(file, 'r', encoding = 'utf-8'):
        tweets.append(json.loads(line))

total_num_of_tweets = len(tweets)
#pprint(tweets[0])
#pprint(tweets[-1])

**Ответ на задание 1:**

In [72]:
print(total_num_of_tweets)

2556


### Задание 2

Информация об удалённых твитах устроена особым образом: при конвертации из json в объект питона это будет словарь с одним ключом - *delete* - это мы и можем проверить, используя метод *.keys()* словарей. В переменную *num_of_deleted_tweets* мы и записали количество удалённых твитов (**362**), которое потом разделим на общее число твитов, чтобы получить ответ.

In [73]:
num_of_deleted_tweets = 0
for tweet in tweets:
    if list(tweet.keys())[0] == 'delete':
        num_of_deleted_tweets += 1

**Ответ на задание 2:** (округлен самим питоном)

In [74]:
print('Процент удалённых твитов: {:%}'.format(num_of_deleted_tweets / total_num_of_tweets))

Процент удалённых твитов: 14.162754%


### Задание 3



# **TODO**: Добавить описание и нормальное название языка и PEP8!!!

In [75]:
popular_langs_of_tweets = Counter()
for tweet in tweets:
    if 'lang' in tweet.keys():
        popular_langs_of_tweets[tweet['lang']] += 1

**Ответ на задание 3:** языки по убыванию по популярности с количеством твитов на них (20 штук, как и советовали в tg)

In [76]:
for lang, num in popular_langs_of_tweets.most_common(20):
    if lang != 'und':
        print('{}\t{}'.format(lang, num))

en	719
ja	438
es	173
ko	149
th	123
ar	119
in	71
pt	69
fr	35
tr	30
tl	29
hi	23
ru	15
fa	8
zh	8
ca	7
ur	6
nl	6
de	6


Твиты с неизвестным языком не учитываются. Их *117*

In [77]:
print(popular_langs_of_tweets['und'])

117


# Задание 4



Чтобы найти число пользователей, которые сделали 2+ твита, создадим список пользователей *users* и проходя список твитов будем в *users* вносить маленький список из двух элементов: уникального id и количество твитов человека с этим id. Когда мы встречаем user'а в первый раз, внесён будет список `[id, 1]`. Поскольку это список списков и все вложеные списки имеют одинаковую длину, то проверять существование id удобно применяя функцию `zip()` к `*users` - отдельным маленьким спискам. Такое применение даёт два списка одинаковой длины, в первом из которых - все id на текущий момент, во втором - количества твитов. id и количества твитов находятся в таких списках на одинаковых местах, что позволяет, (если такой id уже есть), увеличить на 1 количество твитов во вложенном списке исходного списка *users*, используя индекс id в разбитом списке.

В самом начале, условием `if list(tweet.keys())[0] != 'delete'` конечно проверяется, чтобы это не был удалённый твит.

In [100]:
users = list()

for tweet in tweets:
    if list(tweet.keys())[0] != 'delete':
        id = tweet['user']['id']
        if users:
            ids = list(zip(*users))[0]
            if id in ids:
                users[ids.index(id)][1] += 1
            else:
                users.append([id, 1])
        else:
            users.append([id, 1])

Ответ вычисляется просто: из общего количества пользователей нужно вычесть количество пользователей, у которых ровно один твит (0 твитов и меньше быть не может, мы вносили так, чтобы их было 1+), что также делается с помощью `zip(*users)`, но на этот раз нас интересует второй разделённый список - список количеств. Итак, **ответ на задание 4:** 25

In [79]:
more_than_one_tweet = len(users) - list(zip(*users))[1].count(1)
print(more_than_one_tweet)

25


# Задание 5

Для каждого твита (если он не удалённый), если список *hashtags* не пуст, то обойдем все словари в нём (каждый словарь описывает один хэштег), и достанем из каждого словаря текст хэштега. Для начала внесём эти хэштеги в список. Позже к нему мы применим Counter.

In [102]:
hashtags_list = list()

for tweet in tweets:
    if list(tweet.keys())[0] != 'delete':
        hashtags = tweet['entities']['hashtags']
        if hashtags:
            for hashtag in hashtags:
                hashtags_list.append(hashtag['text'])

**Ответ на задание 5:** *не выравнивается, извините(*

In [97]:
print('{:<30s}{:>5s}'.format('Hashtag', 'Count'))
for hashtag, count in Counter(hashtags_list).most_common(20):
    print(u'#{:<25s}{:5>d}'.format(hashtag, count))

Hashtag                       Count
#BTS                      17
#방탄소년단                    13
#AMAs                     11
#人気投票ガチャ                  8
#태형                       7
#뷔                        6
#BTSinChicago             5
#BTSLoveYourselfTour      5
#오늘의방탄                    5
#PledgeForSwachhBharat    5
#MPN                      5
#PCAs                     4
#V                        4
#시카고1회차공연                 4
#เป๊กผลิตโชค              4
#JIMIN                    4
#running                  3
#NCT                      3
#지민                       3
#WajahmuPlastik           3


# Задание 6

In [106]:
tweet_texts = list()
for tweet in tweets:
    if list(tweet.keys())[0] != 'delete' and not 'retweeted_status' in tweet and not 'quoted_status_id' in tweet:
        tweet_texts.append(tweet['text'])

1072
